# 中介分析

### 黄荣贵（复旦大学社会学系）

## 贝叶斯中介分析：单层次回归

通过贝叶斯方法计算上述中介分析可以得到中介效应的分布，从而可以直接计算中介效应的置信区间，而无需依赖对中介效应抽样分布的假定（详见Yuan and MacKinnon. (2009). Bayesian Mediation Analysis. Psychol Methods. 14(4): 301–322. ）。我们可以使用rjags程序包进行计算。

首先，我们使用BUGS/JAGS语言编写上述模型，并将模型以一个字符对象的形式保存起来。在下面的模型设定中：
* N表示样本量
* 在BUGS语言中，~ 表示分布，<- 则表示确定性关系
* 在BUGS语言中， dnorm(正态分布)的第二个参数是precision，是方差的倒数
* 在BUGS语言中，dgamma表示gamma分布，该分布描述方差先验分布，a和b这两个参数设置为0.001 意味着无信息先验分布
* 通过定义theta =  beta_mx * beta_m，我们可以得到中介效应

In [1]:
mediation_model <- "
model {
   for(i in 1:N)
   {
      m[i] ~ dnorm(mean.m[i], prec.m)
      mean.m[i] <- alpha_m + beta_mx * x[i]
      y[i] ~ dnorm(mean.y[i], prec.y)
      mean.y[i] <- alpha + beta_m * m[i] + beta_x * x[i]
   }
   alpha_m ~ dnorm(0, 1.0E-6)
   beta_mx ~ dnorm(0, 1.0E-6)
   alpha ~ dnorm(0, 1.0E-6)
   beta_m ~ dnorm(0, 1.0E-6)
   beta_x ~ dnorm(0, 1.0E-6)
   # dgamma (a, b) is a gamma distribution with the shape parameter a and
   # inverse scale parameter b.
   prec.y ~ dgamma(0.001, 0.001)
   prec.m ~ dgamma(0.001, 0.001)
   theta <-  beta_mx * beta_m
}
"

第二，将所需要的数据构建成一个列表。

第三，加载rjags程序包，调用jags.model生成一个BUGS模型。值得注意的是，jags.model的第一个参数是文本连接（或保存在电脑中的文本文件），因此我们需要通过textConnect函数将上述字符格式的BUGS模型定义转换为文本连接。

第四，通过update函数，开始更新模型。

第五，通过coda.samples函数抽样，得到我们所需要的信息（在这里例子中，我们关心的是theta）。

最后，通过summary函数和plot函数得到theta的分布。

In [2]:
data(Prestige, package="carData")

DL <- list(N = nrow(Prestige),
           y = Prestige$prestige,
           x = Prestige$education,
           m = Prestige$income
          )

library(rjags)
jags <- jags.model(textConnection(mediation_model), data=DL,n.chains=4, n.adapt=100)
update(jags, 5000)
samps <- coda.samples(jags, c("theta"), 2000)
summary(samps)
# plot(samps)

Loading required package: coda
Linked to JAGS 4.2.0
Loaded modules: basemod,bugs


Compiling model graph
   Resolving undeclared variables
   Allocating nodes
Graph information:
   Observed stochastic nodes: 204
   Unobserved stochastic nodes: 7
   Total graph size: 818

Initializing model




Iterations = 5001:7000
Thinning interval = 1 
Number of chains = 4 
Sample size per chain = 2000 

1. Empirical mean and standard deviation for each variable,
   plus standard error of the mean:

          Mean             SD       Naive SE Time-series SE 
      0.988648       0.198220       0.002216       0.007387 

2. Quantiles for each variable:

  2.5%    25%    50%    75%  97.5% 
0.6259 0.8516 0.9826 1.1199 1.3947 


从上述图形可知，模型混合得良好（可以进一步通过诊断分析来判断模型是否混合良好），得到的分布可认为接近理论分布。
* 分析结果现实，中介效应的分别略呈右偏态。
* 比较贝叶斯中介分析和Sobel方法的结果可知，Sobel方法略高估了中介效应。

## 贝叶斯中介分析：多层次回归（模拟分析）

贝叶斯中介分析可以容易地延伸至多层次回归的中介分析。对于如下模型：
$$M_{ij} = \alpha_{m,j} + \beta_{mx,j} X_{ij} + \epsilon_{m,ij}$$

$$Y_{ij} = \alpha_j + \beta_{m,j} M_{ij} + \beta_{x,j} X_{ij} + \epsilon_{ij}$$

我们设定上述模型的参数分布，从而可以生成模拟数据集。其中，

$$\alpha_{m,j} \sim\ N(0, 0.6)$$
$$\alpha_{j} \sim\ N(0, 0.4)$$
$$\epsilon_{m,ij} \sim\ N(0, 0.65)$$
$$\beta_{x,j} \sim\ N(0.2, 0.04)$$
$$\epsilon_{ij} \sim\ N(0, 0.45)$$

系数$(\beta_{mx, j}, \beta_{m,j})$对应的期望为$(0.6, 0.6)$，方差分别是$(0.3, 0.4)$，协方差矩阵为0.13。

此外，第一层次有N1=50单位，第二层次有N2=30单位。

下面的命令生成符合上述设定的随机数：

In [3]:
I <- 50 ## the i-th measure for a specific subject
J <- 30 ## the j-th subject
N <- I * J
Xij <- rnorm(n = N, mean = 30, sd = 5)
measures <- rep(1:I, times = J)
subjects <- rep(1:J, each=I)
alpha_mj <- rep( rnorm(J, 0, sqrt(0.6)), each=I)
alpha_j <- rep( rnorm(J, 0, sqrt(0.4)), each=I)
e_mij <- rnorm(N, 0, sqrt(0.65))
beta_xj <- rep( rnorm(J, 0.2, sqrt(0.04)), each=I)
e_ij <- rnorm(N, 0, sqrt(0.45))
library(MASS)
Beta_mxj_Beta_mj <- mvrnorm(n=J, mu=c(0.4, 0.5), Sigma=matrix(c(0.3,0.13,0.13,0.4), ncol=2))
beta_mxj <- rep( Beta_mxj_Beta_mj[,1], each=I)
beta_mj <- rep( Beta_mxj_Beta_mj[,2], each=I)
Mij <- alpha_mj + beta_mxj * Xij + e_mij
Yij <- alpha_j + beta_mj * Mij + beta_xj * Xij + e_ij

接下来，通过BUGS（JAGS）语言拟合上述模型。

In [4]:
med_mlm <- "
model {
  
  ## random coefficients
  for (j in 1:J) {
    # indepdent normal distribution
    alpha_mj[j] ~ dnorm(alpha_mj_mu, prec.alpha_mj_mu)
    alpha_j[j] ~ dnorm(alpha_j_mu, prec.alpha_j_mu)
    beta_xj[j] ~ dnorm(beta_xj_mu, prec.beta_xj_mu)
    # bivariate normal distribution for beta_mxj and beta_mj
    Betas[j, 1:2] ~ dmnorm(Betas_mu[ ], prec.Betas_mu[ , ])
    # note it is dmnorm instead of dnorm 
  }
  
  ## the two models
  for (irow in 1:N) {
    # model of m
    mean.m[irow] <- alpha_mj[ subjects[irow] ]  +  Betas[ subjects[irow] , 1] * x[irow]
    m[irow] ~ dnorm(mean.m[irow], prec.m)
    
    # model of y
    mean.y[irow] <-  alpha_j[ subjects[irow] ] + Betas[ subjects[irow] , 2] * m[irow] + beta_xj[ subjects[irow] ] * x[irow]
    y[irow] ~ dnorm(mean.y[irow], prec.y)
  }
  
  ## priors associated with indepdent random coefficients
  alpha_mj_mu ~ dnorm(0.0, 1.0E-6)
  alpha_j_mu ~ dnorm(0.0, 1.0E-6)
  beta_xj_mu ~ dnorm(0.000001, 1.0E-6) ## use a very small value instead of 0 to monitor ratio
  sigma.alpha_mj_mu ~ dunif(0, 100) ## note: it is dunif instead of runif
  sigma.alpha_j_mu ~ dunif(0, 100)
  sigma.beta_xj_mu ~ dunif(0, 100)
  prec.alpha_mj_mu <- pow(sigma.alpha_mj_mu, -2)
  prec.alpha_j_mu <- pow(sigma.alpha_j_mu, -2)
  prec.beta_xj_mu <- pow(sigma.beta_xj_mu, -2)
  ## alternatively, we can put priors on prec and then calc sigma
  
  ## priors on Betas_mu and prec.Betas_mu
  ## priors on Betas_mu
  Betas_mu[1:2] ~ dmnorm(mean[ ] , prec[,])
  ## specify the values of hypyer-priors
  mean[1] <- 0
  mean[2] <-  0
  prec[1, 1] <-  1.0E-6
  prec[1, 2] <-  0
  prec[2, 1] <-  0
  prec[2, 2] <-  1.0E-6
  ## priors on prec.Betas_mu (vague inverse-Wishart prior)
  prec.Betas_mu[1:2 , 1:2] ~ dwish(Omega[ , ], 2)
  Omega[1, 1] <-  0.001
  Omega[1, 2] <-  0.0
  Omega[2, 1] <-  0.0
  Omega[2, 2] <-  0.001
  
  # Priors for error term pricisions (inverse gamma priors)
  prec.m ~ dgamma(0.001, 0.001)
  prec.y ~ dgamma(0.001, 0.001)
  
  # vague uniform priors for standard deviations of the second level model
  sigma.beta2 ~ dunif(0, 100)
  sigma.taup ~ dunif(0, 100)
  sigma.beta3 ~ dunif(0, 100)
  prec.beta2 <-  1/(sigma.beta2*sigma.beta2)
  prec.taup <-  1/(sigma.taup*sigma.taup)
  prec.beta3 <-  1/(sigma.beta3*sigma.beta3)
  
  ## generated quantities
  var.Betas[1, 1] <-  prec.Betas_mu[2,2] / (prec.Betas_mu[1,1] * prec.Betas_mu[2,2] - prec.Betas_mu[1,2] * prec.Betas_mu[2,1])
  var.Betas[1, 2] <-  - prec.Betas_mu[1,2] / (prec.Betas_mu[1,1] * prec.Betas_mu[2,2] - prec.Betas_mu[1,2] * prec.Betas_mu[2,1])
  var.Betas[2, 1] <-  - prec.Betas_mu[2,1] / (prec.Betas_mu[1,1] * prec.Betas_mu[2,2] - prec.Betas_mu[1,2] * prec.Betas_mu[2,1])
  var.Betas[2, 2] <-  prec.Betas_mu[1,1] / (prec.Betas_mu[1,1] * prec.Betas_mu[2,2] - prec.Betas_mu[1,2] * prec.Betas_mu[2,1])
  
  ##  mediated is the average mediated effect
  mediated <- Betas_mu[1] * Betas_mu[2] + var.Betas[1, 2]
  total <-  Betas_mu[1] * Betas_mu[2] + var.Betas[1, 2] + beta_xj_mu
  ratio <-  mediated / total
}
"

In [5]:
library(rjags) 
mod <- jags.model(textConnection(med_mlm), data=list(N=N, J=J, y = Yij, x=Xij, m=Mij, subjects=subjects) )
update(mod, 10000)

Compiling model graph
   Resolving undeclared variables
   Allocating nodes
Graph information:
   Observed stochastic nodes: 3000
   Unobserved stochastic nodes: 133
   Total graph size: 13772

Initializing model



In [6]:
samp <- coda.samples(mod, variable.names = c("mediated", "total", "ratio", "alpha_j_mu","alpha_mj_mu", "Betas_mu", "beta_xj_mu"), n.iter = 5000)
summary(samp)


Iterations = 11001:16000
Thinning interval = 1 
Number of chains = 1 
Sample size per chain = 5000 

1. Empirical mean and standard deviation for each variable,
   plus standard error of the mean:

               Mean      SD  Naive SE Time-series SE
Betas_mu[1] 0.28805 0.09297 0.0013148       0.001315
Betas_mu[2] 0.37571 0.12674 0.0017924       0.002141
alpha_j_mu  0.20994 0.17479 0.0024718       0.016841
alpha_mj_mu 0.02749 0.22254 0.0031472       0.016197
beta_xj_mu  0.11537 0.03432 0.0004854       0.001647
mediated    0.29518 0.10157 0.0014363       0.001569
ratio       0.70770 0.09632 0.0013622       0.003318
total       0.41056 0.10598 0.0014988       0.001619

2. Quantiles for each variable:

                2.5%      25%     50%    75%  97.5%
Betas_mu[1]  0.10153  0.22636 0.28786 0.3514 0.4684
Betas_mu[2]  0.12029  0.29300 0.37747 0.4595 0.6262
alpha_j_mu  -0.14221  0.09797 0.21454 0.3244 0.5456
alpha_mj_mu -0.42894 -0.11558 0.03086 0.1794 0.4663
beta_xj_mu   0.04826  0.09267 